In [1]:
import pickle
import gzip
import numpy as np

def load_params(file_path):
    with gzip.open(file_path, 'rb') as f:
        params = pickle.load(f, encoding='latin1')  # 'latin1' encoding for compatibility
    return params

params = load_params('params-mhc1-pan.pkl.gz')

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNetConvPoolLayer(nn.Module):
    def __init__(self, filter_shape, poolsize):
        super(LeNetConvPoolLayer, self).__init__()
        self.conv = nn.Conv2d(
            in_channels=filter_shape[1], 
            out_channels=filter_shape[0], 
            kernel_size=(filter_shape[2], filter_shape[3])
        )
        self.pool = nn.MaxPool2d(kernel_size=poolsize)
    
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.pool(x)
        return x

class LogisticRegression(nn.Module):
    def __init__(self, n_in, n_out):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_in, n_out)
    
    def forward(self, x):
        x = torch.sigmoid(self.linear(x))
        return x

class CNN(nn.Module):
    def __init__(self, nkerns, filtsize, poolsize, hidden):
        super(CNN, self).__init__()
        self.layer0 = LeNetConvPoolLayer(filter_shape=nkerns[0], poolsize=poolsize[0])
        self.layer0_output_shape = ((filtsize[0][0] - 1) // poolsize[0][0] + 1, (filtsize[0][1] - 1) // poolsize[0][1] + 1)
        
        self.layer1 = LeNetConvPoolLayer(filter_shape=nkerns[1], poolsize=poolsize[1])
        self.layer1_output_shape = ((self.layer0_output_shape[0] - filtsize[1][0] + 1) // poolsize[1][0], (self.layer0_output_shape[1] - filtsize[1][1] + 1) // poolsize[1][1])
        
        self.fc = LogisticRegression(n_in=nkerns[1][0] * self.layer1_output_shape[0] * self.layer1_output_shape[1], n_out=1)
    
    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [9]:
def set_params(params, model):
    # Set parameters for layer 0
    layer0_conv_w = torch.tensor(params[0], dtype=torch.float32)
    layer0_conv_b = torch.tensor(params[1], dtype=torch.float32)
    model.layer0.conv.weight.data = layer0_conv_w
    model.layer0.conv.bias.data = layer0_conv_b
    
    # Set parameters for layer 1
    layer1_conv_w = torch.tensor(params[2], dtype=torch.float32)
    layer1_conv_b = torch.tensor(params[3], dtype=torch.float32)
    model.layer1.conv.weight.data = layer1_conv_w
    model.layer1.conv.bias.data = layer1_conv_b
    
    # Set parameters for fully connected layer
    fc_w = torch.tensor(params[4], dtype=torch.float32)
    fc_b = torch.tensor(params[5], dtype=torch.float32)
    model.fc.linear.weight.data = fc_w
    model.fc.linear.bias.data = fc_b

model = CNN(
    nkerns=[(nkerns[0], 1, filtsize[0][0], filtsize[0][1]), (nkerns[1], nkerns[0], filtsize[1][0], filtsize[1][1])],
    poolsize=poolsize,
    hidden=hidden
)
set_params(params, model)

NameError: name 'nkerns' is not defined